In [2]:
# Importing libraries & packages
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import time

# Images path
data = "../Data/"
output = "output/"

# Load in imgs
img1 = cv.imread(data+"aau-city-1.jpg")
img2 = cv.imread(data+"aau-city-2.jpg")

Exercise 1

Load two partly overlapping images into OpenCV (aau-city-1 and -2 from Moodle, or take your own pictures)

Extract Harris Corners from both images. You may use the OpenCV function cornerHarris()

Design and implement your own simple corner matching procedure to find the same points in both images
Hint: Look for inspiration in this OpenCV demo: https://docs.opencv.org/master/dc/d0d/tutorial_py_features_harris.html

In [2]:
try:
    # Convert to grayscale
    gray = cv.cvtColor(img1,cv.COLOR_BGR2GRAY)
    gray2 = cv.cvtColor(img2,cv.COLOR_BGR2GRAY)

    # Harris Corner function
    gray = np.float32(gray)
    gray2 = np.float32(gray2)
    dst = cv.cornerHarris(gray, 2, 3, 0.04)
    dst2 = cv.cornerHarris(gray2, 2, 3, 0.04)

    #result is dilated for marking the corners, not important
    dst = cv.dilate(dst,None)
    dst2 = cv.dilate(dst2,None)

    # Threshold for an optimal value, it may vary depending on the image.
    img1[dst>0.01*dst.max()]=[0,0,255]
    img2[dst2>0.01*dst2.max()]=[0,0,255]

    # Display img
    imgs = np.concatenate((img1, img2), axis=1)
    cv.imshow('Detected Corners',imgs)
    k = cv.waitKey(0)
    if k == ord("q"):
        cv.destroyAllWindows()
        exit(1)
except Exception as e:
    print(f"An error occurred: {str(e)}")

In [3]:
def harris_corner_detector(image, k, threshold):
    # Step 1: Calculate derivatives Ix and Iy using Sobel filters
    Ix = cv.Sobel(image, cv.CV_64F, 1, 0, ksize=3)
    Iy = cv.Sobel(image, cv.CV_64F, 0, 1, ksize=3)

    # Step 2: Calculate products Ix*Ix, Iy*Iy, and Ix*Iy
    IxIx = Ix * Ix
    IyIy = Iy * Iy
    IxIy = Ix * Iy

    # Step 3: Apply Gaussian smoothing to the products
    kernel_size = (3,3)
    IxIx = cv.GaussianBlur(IxIx, kernel_size, 0)
    IyIy = cv.GaussianBlur(IyIy, kernel_size, 0)
    IxIy = cv.GaussianBlur(IxIy, kernel_size, 0)

    # Step 4: Compute the Harris corner response function
    det_M = (IxIx * IyIy) - (IxIy * IxIy)
    trace_M = IxIx + IyIy
    response = det_M - k * (trace_M)** 2

    # Step 5: Thresholding and non-maximum suppression
    corner_points = []
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            if response[y, x] > threshold * response.max():
                corner_points.append((x, y))

    return corner_points

In [6]:
# Images path
data = "../Data/"
output = "output/"

# Load image + convert to grayscale
image = cv.imread(data+"aau-city-1.jpg")
gray = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
image2 = cv.imread(data+"aau-city-2.jpg")
gray2 = cv.cvtColor(image2,cv.COLOR_BGR2GRAY)

# Call the Harris corner detector function
corners = harris_corner_detector(gray, k = 0.04, threshold = 0.01)
corners2 = harris_corner_detector(gray2, k = 0.04, threshold = 0.01)

# Draw detected corners on the image
for corner in corners:
    x, y = corner
    cv.circle(image, (x, y), 5, (0,255,0), -1)  # Draw a circle at each corner
for corner2 in corners2:
    x, y = corner2
    cv.circle(image2, (x, y), 5, (0,255,0), -1)  # Draw a circle at each corner

# Display the image with detected corners
disp_img = np.concatenate((image, image2), axis=1)
cv.imshow('Harris Corners', disp_img)
cv.waitKey(0)
cv.destroyAllWindows()

Exercise 2

Change your solution from last exercise to extract SIFT keypoints and descriptors instead of Harris Corners and your own descriptors
Match feature descriptors using the DescriptorMatcher class
Visualize the matches using the drawMatches function

Exercise 3

Use a homography which includes RANSAC to calculate the transformation between the two images (look at findHomograhy() and warpPerspective()) 